# 🌟 Personalized Weekend Planner Demo
This notebook demonstrates how to build a weekend planner using real-time weather based on location and dates.

Technologies used:
- LangGraph
- LangChain Tools
- External APIs: WeatherAPI


In [ ]:
! pip install langchain langgraph dateparser

In [23]:
#Import Dependencies 

import requests
import dateparser
from datetime import datetime, timedelta
import json
from typing import List, Dict, Any, Annotated, Literal
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict
from typing import Any, Dict, List, Optional
import requests

In [24]:
# Load API Keys
key = json.load(open("keys.json"))
WEATHER_API_KEY = key["weather_api"]

In [25]:
# Helper Functions Section

def resolve_dates(dates: List[str]) -> List[datetime.date]:
    resolved = []
    for d in dates:
        parsed = dateparser.parse(d, settings={'PREFER_DATES_FROM': 'future'})
        if parsed:
            resolved.append(parsed.date())
    return resolved

def fetch_weather(location: str, target_date: datetime.date) -> Dict[str, Any]:
    days_diff = (target_date - datetime.now().date()).days
    if days_diff < 0 or days_diff > 14:
        return {"error": "Date out of range (must be within 14 days)"}

    url = f"http://api.weatherapi.com/v1/forecast.json"
    response = requests.get(url, params={
        "key": WEATHER_API_KEY,
        "q": location,
        "days": days_diff + 1,
        "aqi": "no"
    })
    data = response.json()

    for day in data["forecast"]["forecastday"]:
        if day["date"] == target_date.strftime("%Y-%m-%d"):
            return {
                "condition": day["day"]["condition"]["text"],
                "max_temp_c": day["day"]["maxtemp_c"],
                "min_temp_c": day["day"]["mintemp_c"],
                "rain_chance": day["day"]["daily_chance_of_rain"],
                "wind_kph": day["day"]["maxwind_kph"],
                "is_outdoor_friendly": day["day"]["daily_chance_of_rain"] < 30
            }
    return {"error": "Weather data not found for target date"}

# Multi-date assmebler
def generate_week_weather_data(location: str, days: List[str]) -> List[Dict[str, Any]]:
    dates = resolve_dates(days)
    weather_data = []
    for d in dates:
        daily = fetch_weather(location, d)
        weather_data.append({
            "date": d.strftime("%A, %B %d"),
            "raw_date": d.isoformat(),
            "weather": daily
        })
    return weather_data

In [26]:
# Define the week planner tool

@tool
def plan_week(location: str, days: str = "") -> Dict[str, Any]:
    """
    Fetches weather for planning purposes. LLM will use this to decide activities and food.
    """
    if not days.strip():
        today = datetime.now().date()
        day_list = [(today + timedelta(days=i)).strftime("%A") for i in range(7)]
    else:
        day_list = [d.strip() for d in days.split(",")]

    weather_report = generate_week_weather_data(location, day_list)
    return {
        "location": location,
        "forecast": weather_report
    }

In [27]:
# Initialize the LLM with OpenAI GPT-4o
llm = ChatOpenAI(
    api_key=key["openai_api"],
    model="o4-mini-2025-04-16",
    #model="gpt-4o",
)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools([plan_week])

In [28]:
# ToolNode for handling tool execution
tool_node = ToolNode([plan_week])

In [29]:
# Define state type
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [30]:
# Initialize the graph
graph = StateGraph(State)

# Add tool node
graph.add_node("tool_node", tool_node)

In [ ]:
# Define the prompt node
def prompt_node(state: State) -> State:
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}

# Add prompt node to the graph
graph.add_node("prompt_node", prompt_node)

In [32]:
# Define logic to determine next node
def conditional_edge(state: State) -> Literal["tool_node", "__end__"]:
    last_message = state["messages"][-1]
    return "tool_node" if last_message.tool_calls else "__end__"

# Connect nodes based on logic
graph.add_conditional_edges("prompt_node", conditional_edge)
graph.add_edge("tool_node", "prompt_node")

In [33]:
# Set the entry point and compile the graph
graph.set_entry_point("prompt_node")
APP = graph.compile()

In [ ]:
# Using gpt-4o
user_query = "I am spending my weekend in Washington DC. Can you help me plan my time there? I am interested in outdoor activities, if the weather is favourable."

response = llm.invoke(user_query)
print(response.content)

Here’s a flexible two-day outdoor-focused weekend plan for Washington, DC. Adjust timing to match your arrival/departure and the actual forecast. If it looks rainy or too hot/humid, swap in one of the “weather-backup” options at the end.

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––  
SATURDAY  
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––  
08:00–09:00 
• Breakfast at Eastern Market (377 7th St SE)  
  – Grab coffee and a breakfast sandwich or pastries at Astro Coffee or Market Lunch.  

09:00–12:30  
• National Mall by bike or on foot  
  – Rent a Capital Bikeshare bike near Eastern Market.  
  – Roll north along the Mall: U.S. Capitol → Folklife Pavilion → Smithsonian castles → Washington Monument → WWII Memorial → Lincoln Memorial.  
  – Stop for photos and short breaks; the total loop is about 5–6 miles.  

12:30–13:30  
• Lunch  
  – Food trucks on 7th Street NW between Madison & Pennsylvania, or Blue Duck Tavern’s lunch quick-m

In [ ]:
# Using GPT-4o + Agents
user_query = "I am spending my weekend in Washington DC. Can you help me plan my time there? I am interested in outdoor activities, if the weather is favourable."

response = APP.invoke({"messages": [user_query]})
print(response["messages"][-1].content)

Here's the weather forecast for your weekend in Washington DC:

- **Saturday, April 26**: Unfortunately, Saturday is expected to have heavy rain with a max temperature of 21.3°C and a high chance of rain (88%). Winds could reach up to 24.1 kph, making it unsuitable for outdoor activities.
  
- **Sunday, April 27**: The weather improves significantly on Sunday, with sunny skies and no chance of rain. The temperature will reach up to 22.3°C, with mild winds at 23.0 kph, making it perfect for outdoor activities.

### Activity Suggestions:

**Saturday (Indoor-Focused):**
- **Museums**: Consider visiting the Smithsonian Museums, like the National Air and Space Museum or the National Museum of American History.
- **Cafés & Restaurants**: Enjoy some of the local dining experiences in the city. Maybe try a brunch at one of the well-reviewed spots.
- **Theatres/Entertainment**: Look for shows or performances at the Kennedy Center or other local theaters.

**Sunday (Outdoor-Focused):**
- **Natio

In [ ]:
# Using 04-mini + Agents
user_query = "I am spending my weekend in Washington DC. Can you help me plan my time there? I am interested in outdoor activities, if the weather is favourable."

response = APP.invoke({"messages": [user_query]})
print(response["messages"][-1].content)

Here’s a two-day weekend plan for Washington, D.C., tailored to the forecast (Saturday: heavy rain; Sunday: sunny and pleasant).

— SATURDAY (Heavy rain, not ideal for outdoor activities) —
09:30 AM – 12:00 PM  Smithsonian Museums  
  • Start at the National Museum of American History (Penn Q and 14th St NW).  
  • If you prefer science/tech, head to the National Air and Space Museum (Independence Ave).  
  • Both are free; Metro: Smithsonian station (Orange/Blue/Smiththsonian dr).

12:15 PM – 01:30 PM  Lunch in the Mall area  
  • Indoor café options: Mitsitam Cafe (American Indian menu at the National Museum of the American Indian) or Pão de Queijo at the Air and Space Museum.  

01:45 PM – 04:00 PM  National Gallery of Art  
  • East Building for modern art, West Building for classical.  
  • Free; Sculpture Garden café is covered if you need a short break.

04:15 PM – 06:00 PM  U.S. Capitol Visitor Center or Library of Congress  
  • Both offer guided tours indoors—reserve ahead if

 START
  ↓
[prompt_node] (LLM sees conversation so far)
  ↓
(LLM decides to call a tool, e.g., plan_week)
  ↓
[tool_node] (Tool runs, gets results)
  ↓
(Update messages: add tool output to conversation)
  ↓
[prompt_node again] (LLM sees tool result + conversation)
  ↓
(LLM generates final user-facing response)
  ↓
END
